In [ ]:
!pip install transformers accelerate trl bitsandbytes peft
!pip install datasets

In [ ]:
import torch
import os
from google.colab import userdata
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments,GemmaTokenizer
from peft import LoraConfig
from trl import SFTTrainer

import os
os.environ["HF_TOKEN"]=userdata.get("HF_TOKEN")

In [ ]:
model_id="google/gemma-3-270m"
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(model_id,token=os.environ["HF_TOKEN"])
model1=AutoModelForCausalLM.from_pretrained(model_id,token=os.environ["HF_TOKEN"])

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model1 = model1.to(device)

In [ ]:
text="Hi who is manu?"
device="cuda:0"
input=tokenizer(text,return_tensors="pt").to(device)
output=model1.generate(**input,max_new_tokens=50)

In [ ]:
answer=tokenizer.decode(output[0],skip_special_tokens=True)
print(answer)

In [ ]:
os.environ["WANDB_DISABLED"]="false"
lora_config=LoraConfig(
    r=8,
    target_modules=["q_proj","o_proj","k_proj","v_proj","gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM"
)

In [ ]:
from datasets import Dataset, DatasetDict

# Create 10 example records
data = {
    "description": [
        "He is a Gen AI developer working on NLP projects",
        "He is a machine learning engineer building recommendation systems",
        "He is a backend developer integrating AI APIs for clients",
        "He is a deep learning engineer working on computer vision",
        "He is a software engineer creating AI-powered web applications",
        "He is an AI researcher experimenting with generative models",
        "He is a data engineer designing pipelines for ML workflows",
        "He is a prompt engineer optimizing LLM prompts for efficiency",
        "He is a robotics engineer integrating AI into autonomous systems",
        "He is a cloud engineer managing AI infrastructure and deployments",
        "He is an NLP engineer fine-tuning language models for chatbots",
        "He is a software architect designing scalable AI platforms",
        "He is a DevOps engineer automating AI model deployments",
        "He is a computational linguist analyzing language data with AI",
        "He is a researcher teaching models on specialized AI datasets"
    ],
    "person": [
        "Manu", "Manu", "Manu", "Manu", "Manu",
        "Manu", "Manu", "Manu", "Manu", "Manu",
        "Manu", "Manu", "Manu", "Manu", "Manu"
    ]
}

train_dataset = Dataset.from_dict(data)
dataset = DatasetDict({
    "train": train_dataset
})

In [ ]:
def format_func(example):
    return (
        f"Given the description below, name the person.\n"
        f"Description: {example['description']}\n"
        f"Person: {example['person']}"
    )

In [ ]:
trainer = SFTTrainer(
    model=model1,
    train_dataset=dataset["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=2,
        num_train_epochs=20,
        warmup_steps=2,
        max_steps=-1,
        learning_rate=5e-5,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
        logging_dir="./logs",
        report_to=[],
    ),
    peft_config=lora_config,
    formatting_func=format_func,
)

In [ ]:
text="Given the description below, name the person.\n Description: He is a machine learning engineer building recommendation systems"
device="cuda:0"
input=tokenizer(text,return_tensors="pt").to(device)
output = model1.generate(
    **input,
    max_new_tokens=100,
    do_sample=False,
    temperature=0.7,
    top_p=0.9
)

In [ ]:
answer=tokenizer.decode(output[0],skip_special_tokens=True)
print(answer)

In [ ]:
"""
Given the description below, name the person.
 Description: He is a machine learning engineer building recommendation systems
 Person: Manu
"""